<a href="https://colab.research.google.com/github/csiongn/facial-expression-recognition/blob/main/Jumpstart_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ng Choon Siong

**Deep Learning Jumpstart Workshop: Project on facial expression recognition**

---
Data: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

---



*Introduction*

This project makes use of a simple convolutional neural network in order to catagorise images of faces into one of 7 categories (Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral) based on the emotion shown. 

## Connecting to Kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              215KB  2021-03-15 08:28:36            420  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            430  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            368  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            163  
fatiimaezzahra/famous-iconic-women                          Famous Iconic Wo

Importing data

In [ ]:
! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 92% 85.0M/92.0M [00:00<00:00, 95.6MB/s]
100% 92.0M/92.0M [00:00<00:00, 118MB/s] 
 84% 81.0M/96.6M [00:02<00:00, 43.2MB/s]
100% 96.6M/96.6M [00:02<00:00, 45.6MB/s]
  0% 0.00/7.01k [00:00<?, ?B/s]
100% 7.01k/7.01k [00:00<00:00, 6.30MB/s]
 74% 57.0M/77.3M [00:00<00:00, 73.5MB/s]
100% 77.3M/77.3M [00:00<00:00, 122MB/s] 
 41% 8.00M/19.3M [00:00<00:00, 83.0MB/s]
100% 19.3M/19.3M [00:00<00:00, 94.4MB/s]


In [ ]:
!mkdir train
!mkdir test
!unzip train.csv.zip -d train
!unzip test.csv.zip -d test
!rm train.csv.zip
!rm test.csv.zip

Archive:  train.csv.zip
  inflating: train/train.csv         
Archive:  test.csv.zip
  inflating: test/test.csv           


## Using pandas to read dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')

In [ ]:
train_df

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...
28704,2,84 85 85 85 85 85 85 85 86 86 86 87 86 86 91 9...
28705,0,114 112 113 113 111 111 112 113 115 113 114 11...
28706,4,74 81 87 89 95 100 98 93 105 120 127 133 146 1...
28707,0,222 227 203 90 86 90 84 77 94 87 99 119 134 14...


In [ ]:
test_df

,pixels
0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...
1,156 184 198 202 204 207 210 212 213 214 215 21...
2,69 118 61 60 96 121 103 87 103 88 70 90 115 12...
3,205 203 236 157 83 158 120 116 94 86 155 180 2...
4,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...
...,...
7173,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
7174,178 174 172 173 181 188 191 194 196 199 200 20...
7175,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
7176,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [ ]:
train_x_raw = train_df['pixels'].str.split() # Memory Intensive: Extracts pixels attribute string and splits by whitespace
train_y_raw = train_df['emotion']

test_x_raw = test_df['pixels'].str.split()

In [ ]:
train_y_raw

0        0
1        0
2        2
3        4
4        6
        ..
28704    2
28705    0
28706    4
28707    0
28708    4
Name: emotion, Length: 28709, dtype: int64

In [ ]:
train_x_raw = np.array(list(map(lambda lst: list(map(float, lst)), train_x_raw)))
test_x_raw = np.array(list(map(lambda lst: list(map(float, lst)), test_x_raw)))

## Data preprocessing

In [ ]:
length_of_image = 48
batch_size = 64
number_of_classes = 7
epochs = 10
learning_rate = 0.001

In [ ]:
train_x_raw.shape

(28709, 2304)

In [ ]:
# Calculate how much data to use for training and how much for validation
n_training = int(0.9 * train_x_raw.shape[0]) # 90% of training data is used for training
n_validation = train_x_raw.shape[0] - n_training # Remaining training data used for validation

In [ ]:
n_training

25838

In [ ]:
train_x, validation_x = train_x_raw[:n_training,:], train_x_raw[n_training:,:]

In [ ]:
len(train_y_raw)

28709

In [ ]:
train_y, validation_y = train_y_raw[:n_training], train_y_raw[n_training:]

In [ ]:
test_x = test_x_raw

In [ ]:
train_x = train_x.reshape(train_x.shape[0], length_of_image, length_of_image, 1)
validation_x = validation_x.reshape(validation_x.shape[0], length_of_image, length_of_image, 1)
test_x = test_x.reshape(test_x.shape[0], length_of_image, length_of_image, 1)
input_shape = (length_of_image, length_of_image, 1)

In [ ]:
train_x.shape, validation_x.shape, test_x.shape

((25838, 48, 48, 1), (2871, 48, 48, 1), (7178, 48, 48, 1))

Training dataset: Batch dimension (25838), image size (48 x 48 x 1)

Validation dataset: Batch dimension (2871), image size (48 x 48 x 1)

Testing dataset: Batch dimension (7178), image size (48 x 48 x 1)

Currently, each image pixel is represented by an integer in [0, 255]. So we perform normalisation. 

In [ ]:
train_x /= 255.
validation_x /= 255.
test_x /= 255.

## Data exploration

In [ ]:
train_df['emotion']

0        0
1        0
2        2
3        4
4        6
        ..
28704    2
28705    0
28706    4
28707    0
28708    4
Name: emotion, Length: 28709, dtype: int64

In [ ]:
# Frequency distribution of labels in training set
unique_elements, counts_elements = np.unique(train_df['emotion'], return_counts=True)
unique_elements, counts_elements

(array([0, 1, 2, 3, 4, 5, 6]),
 array([3995,  436, 4097, 7215, 4830, 3171, 4965]))

## Setting up tensorflow

In [ ]:
import os

import tensorflow as tf

import tensorflow.keras

from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
train_y = tf.keras.utils.to_categorical(train_y, number_of_classes)
validation_y = tf.keras.utils.to_categorical(validation_y, number_of_classes)

In [ ]:
train_y.shape, validation_y.shape

((25838, 7), (2871, 7))

In [ ]:
Inp=Input(shape=input_shape)
x = Conv2D(32, kernel_size=(2, 2), activation='relu',name = 'Conv_01')(Inp)
x = Conv2D(32, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Dropout(0.5, name = 'Dropout_01')(x)
x = Conv2D(32, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Conv2D(32, (3, 3), activation='relu',name = 'Conv_04')(x)
x = MaxPooling2D(pool_size=(3, 3),name = 'MaxPool_02')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(128, activation='relu',name = 'Dense_01')(x)
x = Dense(128, activation='relu',name = 'Dense_02')(x)
x = Dense(128, activation='relu',name = 'Dense_03')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)
output = Dense(number_of_classes, activation='softmax',name = 'Dense_04')(x)

In [ ]:
model = Model(Inp,output,name='CNN_Model')

In [ ]:
model.summary()

Model: "CNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
Conv_01 (Conv2D)             (None, 47, 47, 32)        160       
_________________________________________________________________
Conv_02 (Conv2D)             (None, 45, 45, 32)        9248      
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (None, 22, 22, 32)        0         
_________________________________________________________________
Dropout_01 (Dropout)         (None, 22, 22, 32)        0         
_________________________________________________________________
Conv_03 (Conv2D)             (None, 20, 20, 32)        9248      
_________________________________________________________________
Conv_04 (Conv2D)             (None, 18, 18, 32)        92

In [ ]:
#optimizer
opt = tf.keras.optimizers.Adam(lr = learning_rate)

model.compile(loss= tf.keras.losses.categorical_crossentropy, #'categorical_crossentropy'
              optimizer= opt,
              metrics=['accuracy'])

In [ ]:
hist = model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks = None,
          validation_data=(validation_x, validation_y))

Epoch 1/10
404/404 [==============================] - 38s 12ms/step - loss: 1.8285 - accuracy: 0.2344 - val_loss: 1.6030 - val_accuracy: 0.3678
Epoch 2/10
404/404 [==============================] - 4s 11ms/step - loss: 1.5981 - accuracy: 0.3685 - val_loss: 1.4324 - val_accuracy: 0.4563
Epoch 3/10
404/404 [==============================] - 4s 11ms/step - loss: 1.4456 - accuracy: 0.4420 - val_loss: 1.3119 - val_accuracy: 0.4953
Epoch 4/10
404/404 [==============================] - 4s 11ms/step - loss: 1.3671 - accuracy: 0.4747 - val_loss: 1.2495 - val_accuracy: 0.5193
Epoch 5/10
404/404 [==============================] - 4s 11ms/step - loss: 1.2942 - accuracy: 0.4994 - val_loss: 1.2276 - val_accuracy: 0.5253
Epoch 6/10
404/404 [==============================] - 4s 11ms/step - loss: 1.2516 - accuracy: 0.5229 - val_loss: 1.2102 - val_accuracy: 0.5423
Epoch 7/10
404/404 [==============================] - 4s 11ms/step - loss: 1.2096 - accuracy: 0.5435 - val_loss: 1.1594 - val_accuracy: 0.558

## Model prediction

In [ ]:
predictions = model.predict(validation_x)

In [ ]:
predictions[:5]

array([[3.4398139e-01, 9.8881051e-02, 2.2806989e-01, 2.5771835e-03,
        2.6521105e-01, 1.1329491e-02, 4.9950007e-02],
       [6.7370682e-05, 1.3458423e-08, 7.5477568e-05, 9.9866986e-01,
        1.3440236e-04, 9.2617207e-05, 9.6032617e-04],
       [1.3908984e-01, 2.3343125e-02, 2.7862144e-01, 8.0544101e-03,
        3.0099747e-01, 4.5158293e-02, 2.0473546e-01],
       [1.3716668e-01, 1.0118799e-02, 2.0357925e-01, 9.8499963e-03,
        4.7100139e-01, 1.1953019e-02, 1.5633090e-01],
       [2.9051468e-01, 1.3358365e-01, 3.4705257e-01, 3.6849253e-02,
        1.3443510e-01, 4.0600691e-02, 1.6963985e-02]], dtype=float32)

In [ ]:
validation_y[:5]

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.]], dtype=float32)